In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['trusted_zone']
collection = db['juegos_steam']
data = collection.find()
for doc in data:
    print(doc)

{'_id': ObjectId('685e9d061b55cd7ad4c0f3bd'), 'name': 'DARK SOULS™: REMASTERED', 'detailed_description': 'Then, there was fire. Re-experience the critically acclaimed, genre-defining game that started it all. Beautifully remastered, return to Lordran in stunning high-definition detail running at 60fps. \r\nDark Souls Remastered includes the main game plus the Artorias of the Abyss DLC.\r\n\r\nKey features:\r\n• Deep and Dark Universe \r\n• Each End is a New Beginning\r\n• Gameplay Richness and Possibilities\r\n• Sense of Learning, Mastering and Accomplishment\r\n• The Way of the Multiplayer (up to 6 players with dedicated servers)', 'about_the_game': 'Then, there was fire. Re-experience the critically acclaimed, genre-defining game that started it all. Beautifully remastered, return to Lordran in stunning high-definition detail running at 60fps. \r\nDark Souls Remastered includes the main game plus the Artorias of the Abyss DLC.\r\n\r\nKey features:\r\n• Deep and Dark Universe \r\n• Ea

In [2]:
pip install pymongo pandas scikit-learn textblob numpy

  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   --------------------------------- ------ 8.9/10.7 MB 46.0 MB/s eta 0:00:01
   ---------------------------------------- 10.7/10.7 MB 37.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   --------------------------------------- 624.3/624.3 kB 11.7 MB/s eta 0:00:00
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 77.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ----------------- ---------------------- 17.3/38.6 MB 83.9 MB/s eta 0:00:01
   ----------------------------------- ---- 34.6/38.6 MB 84.6 MB/s eta 0:00:01
   

In [3]:
# Importar librerías necesarias
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
import numpy as np

# Conexión a MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['trusted_zone']

# Acceder a las colecciones
juegos_collection = db['juegos_steam']
reviews_collection = db['steam_reviews']


In [9]:

# 1. Obtener datos y convertir a DataFrame
juegos_data = list(juegos_collection.find())
juegos_df = pd.DataFrame(juegos_data)


In [10]:

# 2. Transformaciones textuales (TF-IDF para campos de texto)
text_fields = ['name', 'detailed_description', 'about_the_game', 'short_description']
missing_text_fields = [field for field in text_fields if field not in juegos_df.columns]
if missing_text_fields:
    print(f"ADVERTENCIA: Las siguientes columnas de texto no se encontraron en juegos_df: {missing_text_fields}")
    text_fields = [field for field in text_fields if field in juegos_df.columns]
    if not text_fields:
        print("ERROR: No quedan campos de texto válidos para TF-IDF. Omitiendo transformación textual.")
        text_df = pd.DataFrame()
    else:
        print(f"INFO: Procesando TF-IDF con los campos de texto disponibles: {text_fields}")

if text_fields:
    juegos_df_text_subset = juegos_df[text_fields].fillna('').astype(str)
    text_combined = juegos_df_text_subset.apply(lambda x: ' '.join(x), axis=1)
    tfidf = TfidfVectorizer(max_features=500)
    text_vectors = tfidf.fit_transform(text_combined)
    text_df = pd.DataFrame(text_vectors.toarray(), columns=tfidf.get_feature_names_out())
else:
    text_df = pd.DataFrame()


In [11]:

# 3. Transformaciones categóricas (One-Hot Encoding)
cat_fields = ['supported_languages', 'platforms', 'categories']
encoded_dfs = []
juegos_df = juegos_df.reset_index(drop=True) # Asegurar índice continuo

for field in cat_fields:
    if field not in juegos_df.columns:
        print(f"ADVERTENCIA: La columna categórica '{field}' no se encontró en juegos_df. Se omitirá.")
        continue

    temp_field_series = juegos_df[field].apply(lambda x: x if isinstance(x, list) else [x] if pd.notnull(x) else [])
    if temp_field_series.apply(lambda x: len(x)).sum() == 0:
        print(f"INFO: La columna '{field}' está vacía o solo contiene nulos/listas vacías. Omitiendo One-Hot Encoding para esta columna.")
        continue

    temp_df_for_explode = juegos_df[['_id']].copy()
    temp_df_for_explode[field] = temp_field_series
    temp_df_for_explode = temp_df_for_explode.explode(field).dropna(subset=[field])

    if temp_df_for_explode.empty:
        print(f"INFO: '{field}' resultó en un DataFrame vacío después de explode/dropna. Omitiendo One-Hot Encoding.")
        continue

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded = encoder.fit_transform(temp_df_for_explode[[field]].astype(str))
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([field]))
    encoded_df = pd.concat([temp_df_for_explode.reset_index(drop=True)[['_id']], encoded_df], axis=1)
    encoded_df_agg = encoded_df.groupby('_id').sum().reset_index()
    encoded_dfs.append(encoded_df_agg)

if encoded_dfs:
    cat_df = encoded_dfs[0]
    for df in encoded_dfs[1:]:
        cat_df = cat_df.merge(df, on='_id', how='outer')
    cat_df = cat_df.fillna(0)
else:
    cat_df = pd.DataFrame({'_id': juegos_df['_id'].unique()})


In [12]:

# 4. Alinear juegos_df con cat_df usando _id
juegos_df = juegos_df.merge(cat_df, on='_id', how='left')


In [13]:

# 5. Transformaciones numéricas
if 'price_overview' in juegos_df.columns:
    juegos_df['price_overview.final'] = pd.to_numeric(
        juegos_df['price_overview'].apply(lambda x: x.get('final', 0) if isinstance(x, dict) else 0),
        errors='coerce'
    )
else:
    juegos_df['price_overview.final'] = 0
    print("ADVERTENCIA: La columna 'price_overview' no se encontró. 'price_overview.final' se estableció en 0.")

if 'required_age' in juegos_df.columns:
    juegos_df['required_age'] = pd.to_numeric(juegos_df['required_age'], errors='coerce').fillna(0)
else:
    juegos_df['required_age'] = 0
    print("ADVERTENCIA: La columna 'required_age' no se encontró. Se estableció en 0.")


In [14]:

# 6. Transformaciones temporales (calcular antigüedad)
current_date = datetime(2025, 6, 27)
if 'release_date' in juegos_df.columns:
    juegos_df['release_date'] = pd.to_datetime(juegos_df['release_date'], errors='coerce')
    juegos_df['age_days'] = (current_date - juegos_df['release_date']).dt.days.fillna(0)
else:
    juegos_df['release_date'] = pd.NaT
    juegos_df['age_days'] = 0
    print("ADVERTENCIA: La columna 'release_date' no se encontró. 'age_days' se estableció en 0.")


In [15]:

# 7. Combinar todas las transformaciones
if text_df.empty:
    columns_to_drop = text_fields + cat_fields
    final_juegos_processed = juegos_df.drop(columns=[col for col in columns_to_drop if col in juegos_df.columns], errors='ignore')
else:
    juegos_df_temp = juegos_df.reset_index(drop=True)
    text_df_temp = text_df.reset_index(drop=True)

    if len(juegos_df_temp) != len(text_df_temp):
        print("ERROR: El número de filas de juegos_df y text_df no coincide. No se puede concatenar directamente.")
        final_juegos_processed = juegos_df_temp
    else:
        columns_to_drop = text_fields + cat_fields
        final_juegos_processed = pd.concat([juegos_df_temp.drop(columns=[col for col in columns_to_drop if col in juegos_df_temp.columns], errors='ignore'), text_df_temp], axis=1)

print("\n--- Resultados finales procesados (juegos_processed) ---")
if 'final_juegos_processed' in locals():
    print(final_juegos_processed.head())
    print("\nInformación general de juegos_processed:")
    print(final_juegos_processed.info())
else:
    print("El DataFrame final 'final_juegos_processed' no se pudo crear debido a errores anteriores.")



--- Resultados finales procesados (juegos_processed) ---
                        _id  \
0  685e9d061b55cd7ad4c0f3bd   
1  685e9d061b55cd7ad4c0f3be   
2  685e9d061b55cd7ad4c0f3bf   
3  685e9d061b55cd7ad4c0f3c0   
4  685e9d061b55cd7ad4c0f3c1   

                                        legal_notice  \
0  Dark Souls™: Remastered & ©BANDAI NAMCO Entert...   
1  DARK SOULS® III & ©BANDAI NAMCO Entertainment ...   
2  DARK SOULS™ II: Scholar of the First Sin & ©20...   
3  ELDEN RING™ & ©BANDAI NAMCO Entertainment Inc....   
4                      ⓒ NEOWIZ All rights reserved.   

                                     pc_requirements  \
0  {'minimum': 'Minimum:Requires a 64-bit process...   
1  {'minimum': 'Minimum:OS *: Windows 7 SP1 64bit...   
2  {'minimum': 'Minimum:OS *: Windows 7 SP1 64bit...   
3  {'minimum': 'Minimum:Requires a 64-bit process...   
4  {'minimum': 'Minimum:Requires a 64-bit process...   

                                    mac_requirements  \
0  {'minimum': '{"minimum

In [16]:

# Obtener datos y convertir a DataFrame
reviews_data = list(reviews_collection.find())
reviews_df = pd.DataFrame(reviews_data)

# Análisis de sentimiento básico
from textblob import TextBlob
if 'review_clean' in reviews_df.columns:
    reviews_df['sentiment_score'] = reviews_df['review_clean'].apply(lambda x: TextBlob(str(x)).sentiment.polarity if pd.notnull(x) else 0)
else:
    reviews_df['sentiment_score'] = 0
    print("ADVERTENCIA: La columna 'review_clean' no se encontró en reviews_df. El cálculo de sentimiento se estableció en 0.")



In [20]:
# Agregar métricas por juego usando recommendationid
if 'recommendationid' in reviews_df.columns:
    columns_to_agg = [
        'num_games_owned',
        'num_reviews',
        'playtime_at_review',
        'playtime_forever',
        'voted_up',
        'votes_up'
    ]
    agg_dict = {}
    for col in columns_to_agg:
        if col in reviews_df.columns:
            if col == 'num_reviews' or col == 'votes_up':
                agg_dict[col] = 'sum'
            else:
                agg_dict[col] = 'mean'
        else:
            print(f"ADVERTENCIA: La columna '{col}' no se encontró en reviews_df y no se incluirá en la agregación.")

    if agg_dict:
        agg_metrics = reviews_df.groupby('recommendationid').agg(agg_dict).reset_index()

        # --- AQUI ES DONDE INTEGRAS LA CONVERSION DEL TIPO DE DATO ---
        # Convertir 'recommendationid' a numérico (int64)
        # Esto es crucial porque en agg_metrics, 'recommendationid' viene del groupby
        # y podría ser 'object' si los IDs originales no eran estrictamente numéricos o estaban mezclados.
        agg_metrics['recommendationid'] = pd.to_numeric(
            agg_metrics['recommendationid'], errors='coerce'
        ).astype('Int64') # Usar 'Int64' (con I mayúscula) para manejar NaNs si los hay

        # Opcional: Eliminar filas con NaN en 'recommendationid' si no se pudieron convertir
        # Esto previene problemas si algunos 'recommendationid' no eran números válidos
        agg_metrics.dropna(subset=['recommendationid'], inplace=True)
        # --- FIN DE LA INTEGRACION ---

    else:
        print("No hay columnas válidas para realizar la agregación de reviews.")
        agg_metrics = pd.DataFrame()
else:
    print("ERROR: La columna 'recommendationid' no se encontró en reviews_df. No se puede agrupar reviews.")
    agg_metrics = pd.DataFrame()

print("\n--- Datos procesados de steam_reviews agregados (agg_metrics) ---")
print(agg_metrics.head())


# --- Y también asegúrate de que 'appid' en final_juegos_processed sea 'Int64' antes del merge ---
# Esto es para asegurar la compatibilidad, aunque ya es int64, una conversión a Int64 permite NaNs
if 'final_juegos_processed' in locals() and not final_juegos_processed.empty and 'appid' in final_juegos_processed.columns:
    final_juegos_processed['appid'] = pd.to_numeric(
        final_juegos_processed['appid'], errors='coerce'
    ).astype('Int64')
    # Opcional: Eliminar filas con NaN en 'appid' si no se pudieron convertir
    final_juegos_processed.dropna(subset=['appid'], inplace=True)


# Unir con juegos procesados usando appid y recommendationid
merged_data = pd.DataFrame()
if 'final_juegos_processed' in locals() and not final_juegos_processed.empty and \
   'agg_metrics' in locals() and not agg_metrics.empty: # Verificación adicional para agg_metrics
    merged_data = pd.merge(final_juegos_processed, agg_metrics, left_on='appid', right_on='recommendationid', how='left')
    print("\n--- Datos combinados (merged_data) ---")
    print(merged_data.head())
else:
    print("\nNo se pudo realizar la unión: 'final_juegos_processed' o 'agg_metrics' están vacíos o no definidos.")

# ... (El resto de tu código para guardar en MongoDB) ...


ADVERTENCIA: La columna 'num_games_owned' no se encontró en reviews_df y no se incluirá en la agregación.
ADVERTENCIA: La columna 'num_reviews' no se encontró en reviews_df y no se incluirá en la agregación.
ADVERTENCIA: La columna 'playtime_at_review' no se encontró en reviews_df y no se incluirá en la agregación.
ADVERTENCIA: La columna 'playtime_forever' no se encontró en reviews_df y no se incluirá en la agregación.

--- Datos procesados de steam_reviews agregados (agg_metrics) ---
   recommendationid  voted_up  votes_up
0          10000058       1.0         0
1         100001792       1.0         0
2         100002056       1.0         0
3          10000235       1.0         0
4          10000279       1.0         0

--- Datos combinados (merged_data) ---
                        _id  \
0  685e9d061b55cd7ad4c0f3bd   
1  685e9d061b55cd7ad4c0f3be   
2  685e9d061b55cd7ad4c0f3bf   
3  685e9d061b55cd7ad4c0f3c0   
4  685e9d061b55cd7ad4c0f3c1   

                                        le

In [21]:

# Unir con juegos procesados usando appid y recommendationid
merged_data = pd.DataFrame() # Inicializar por si no se puede fusionar
if 'final_juegos_processed' in locals() and not final_juegos_processed.empty and not agg_metrics.empty:
    merged_data = pd.merge(final_juegos_processed, agg_metrics, left_on='appid', right_on='recommendationid', how='left')
    print("\n--- Datos combinados (merged_data) ---")
    print(merged_data.head())
else:
    print("\nNo se pudo realizar la unión: 'final_juegos_processed' o 'agg_metrics' están vacíos o no definidos.")




--- Datos combinados (merged_data) ---
                        _id  \
0  685e9d061b55cd7ad4c0f3bd   
1  685e9d061b55cd7ad4c0f3be   
2  685e9d061b55cd7ad4c0f3bf   
3  685e9d061b55cd7ad4c0f3c0   
4  685e9d061b55cd7ad4c0f3c1   

                                        legal_notice  \
0  Dark Souls™: Remastered & ©BANDAI NAMCO Entert...   
1  DARK SOULS® III & ©BANDAI NAMCO Entertainment ...   
2  DARK SOULS™ II: Scholar of the First Sin & ©20...   
3  ELDEN RING™ & ©BANDAI NAMCO Entertainment Inc....   
4                      ⓒ NEOWIZ All rights reserved.   

                                     pc_requirements  \
0  {'minimum': 'Minimum:Requires a 64-bit process...   
1  {'minimum': 'Minimum:OS *: Windows 7 SP1 64bit...   
2  {'minimum': 'Minimum:OS *: Windows 7 SP1 64bit...   
3  {'minimum': 'Minimum:Requires a 64-bit process...   
4  {'minimum': 'Minimum:Requires a 64-bit process...   

                                    mac_requirements  \
0  {'minimum': '{"minimum":"Minimum:","reco

In [23]:
# --- SECCIÓN PARA GUARDAR EN 'steam_games_all' CON VALIDACIÓN MEJORADA ---

print("\n--- Iniciando el proceso de guardado en la colección 'steam_games_all' ---")

new_collection_name = 'steam_games_all'
steam_games_all_collection = db[new_collection_name]

# Opcional: Eliminar todos los documentos existentes en la nueva colección antes de insertar nuevos
steam_games_all_collection.delete_many({})
print(f"Colección '{new_collection_name}' vaciada (si contenía documentos).")


# Validar si el DataFrame final tiene datos antes de intentar guardarlo
if not merged_data.empty:
    print(f"Detectados {len(merged_data)} documentos para insertar en '{new_collection_name}'.")
    # Convertir DataFrame a una lista de diccionarios
    combined_records = merged_data.to_dict('records')

    # Insertar los documentos en la colección
    try:
        steam_games_all_collection.insert_many(combined_records)
        print(f"¡Éxito! Se insertaron {len(combined_records)} documentos combinados en '{new_collection_name}'.")
    except Exception as e:
        print(f"ERROR: Fallo al insertar documentos combinados en '{new_collection_name}': {e}")
else:
    print(f"El DataFrame 'merged_data' está vacío. No se guardarán datos en '{new_collection_name}'.")

print("\nProceso de guardado completado.")


--- Iniciando el proceso de guardado en la colección 'steam_games_all' ---
Colección 'steam_games_all' vaciada (si contenía documentos).
Detectados 10 documentos para insertar en 'steam_games_all'.
¡Éxito! Se insertaron 10 documentos combinados en 'steam_games_all'.

Proceso de guardado completado.


In [35]:
# --- 1. Leer datos de YouTube ---

# Leer la colección 'video_youtube'
print("--- Leyendo la colección 'video_youtube' ---")
try:
    videos_data = list(db['video_youtube'].find()) # Asegúrate de que el nombre de la colección es correcto
    videos_df = pd.DataFrame(videos_data)
    print(f"Se cargaron {len(videos_df)} documentos de 'video_youtube'.")
    if not videos_df.empty:
        print("Columnas de videos_df:", videos_df.columns.tolist())
        print("Primeras 3 filas de videos_df:\n", videos_df.head(3))
        print("Tipos de datos de videos_df:\n", videos_df.info(verbose=False))
    else:
        print("La colección 'video_youtube' está vacía.")
except Exception as e:
    print(f"ERROR al leer 'video_youtube': {e}")
    videos_df = pd.DataFrame() # Inicializar como DataFrame vacío en caso de error


# Leer la colección 'comentarios_youtube'
print("\n--- Leyendo la colección 'comentarios_youtube' ---")
try:
    comments_data = list(db['comentarios_youtube'].find()) # Asegúrate de que el nombre de la colección es correcto
    comments_df = pd.DataFrame(comments_data)
    print(f"Se cargaron {len(comments_df)} documentos de 'comentarios_youtube'.")
    if not comments_df.empty:
        print("Columnas de comments_df:", comments_df.columns.tolist())
        print("Primeras 3 filas de comments_df:\n", comments_df.head(3))
        print("Tipos de datos de comments_df:\n", comments_df.info(verbose=False))
    else:
        print("La colección 'comentarios_youtube' está vacía.")
except Exception as e:
    print(f"ERROR al leer 'comentarios_youtube': {e}")
    comments_df = pd.DataFrame() # Inicializar como DataFrame vacío en caso de error


--- Leyendo la colección 'video_youtube' ---
Se cargaron 91 documentos de 'video_youtube'.
Columnas de videos_df: ['_id', 'videoId', 'transcription']
Primeras 3 filas de videos_df:
                         _id      videoId  \
0  685e9d49164dd170f6984ef7  -h7oRpfHyyg   
1  685e9d49164dd170f6984ef9  0XF4VBXPuhE   
2  685e9d49164dd170f6984efb  1BxOeeKU-Zs   

                                       transcription  
0  [Music] The last time we spoke about Lies of P...  
1  [Music] if you think you can change asa's fate...  
2  and we're back with another episode of before ...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Columns: 3 entries, _id to transcription
dtypes: object(3)
memory usage: 2.3+ KB
Tipos de datos de videos_df:
 None

--- Leyendo la colección 'comentarios_youtube' ---
Se cargaron 160023 documentos de 'comentarios_youtube'.
Columnas de comments_df: ['_id', 'videoId', 'author', 'text', 'text_clean', 'publishedAt', 'likeCount']
Primeras 3 filas de com

In [36]:

# --- 2. Procesamiento y Preparación para la Unión ---

# --- Estandarizar 'videoId' a string en ambos DataFrames ---
print("\n--- Estandarizando 'videoId' a string para la unión ---")
if not videos_df.empty and 'videoId' in videos_df.columns:
    videos_df['videoId'] = videos_df['videoId'].astype(str)
    print(f"Tipo de dato de 'videoId' en videos_df (después de estandarizar): {videos_df['videoId'].dtype}")
else:
    print("Advertencia: 'videoId' no encontrado o videos_df está vacío, no se estandarizó 'videoId' en videos_df.")

if not comments_df.empty and 'videoId' in comments_df.columns:
    comments_df['videoId'] = comments_df['videoId'].astype(str)
    print(f"Tipo de dato de 'videoId' en comments_df (después de estandarizar): {comments_df['videoId'].dtype}")
else:
    print("Advertencia: 'videoId' no encontrado o comments_df está vacío, no se estandarizó 'videoId' en comments_df.")
print("-" * 60)


# --- Análisis de sentimiento para Transcripciones de Video (desde videos_df) ---
if 'transcription' in videos_df.columns and not videos_df.empty:
    print("\n--- Realizando análisis de sentimiento en transcripciones de YouTube (desde videos_df) ---")
    videos_df['transcription_sentiment_score'] = videos_df['transcription'].apply(
        lambda x: TextBlob(str(x)).sentiment.polarity if pd.notnull(x) else 0
    )
    print("Columna 'transcription_sentiment_score' añadida a videos_df.")
    print(videos_df[['videoId', 'transcription', 'transcription_sentiment_score']].head())
else:
    print("ADVERTENCIA: La columna 'transcription' no se encontró o videos_df está vacío. No se realizó análisis de sentimiento en transcripciones.")
    videos_df['transcription_sentiment_score'] = 0 # Asegurarse de que la columna exista con un valor por defecto


# --- Análisis de sentimiento para Comentarios Individuales (desde comments_df) ---
if 'text' in comments_df.columns and not comments_df.empty: # <-- AQUI ESTÁ EL CAMBIO CLAVE: USAR 'text'
    print("\n--- Realizando análisis de sentimiento en comentarios de YouTube (desde comments_df) ---")
    comments_df['comment_sentiment_score'] = comments_df['text'].apply( # <-- AQUI ESTÁ EL CAMBIO CLAVE: USAR 'text'
        lambda x: TextBlob(str(x)).sentiment.polarity if pd.notnull(x) else 0
    )
    print("Columna 'comment_sentiment_score' añadida a comments_df.")
    print(comments_df[['videoId', 'text', 'comment_sentiment_score']].head())

    # Agregación de comentarios por videoId (si un video tiene múltiples comentarios)
    # Si quieres una métrica de sentimiento por video (no por comentario individual), agrégala aquí:
    print("\n--- Agregando métricas de sentimiento de comentarios por 'videoId' ---")
    aggregated_youtube_comments = comments_df.groupby('videoId').agg(
        avg_comment_sentiment=('comment_sentiment_score', 'mean'),
        total_comments=('comment_sentiment_score', 'count') # Contar el número de comentarios con score
    ).reset_index()
    print("Métricas agregadas de comentarios por videoId:\n", aggregated_youtube_comments.head())

else:
    print("ADVERTENCIA: La columna 'text' no se encontró o comments_df está vacío. No se realizó análisis de sentimiento en comentarios.")
    comments_df['comment_sentiment_score'] = 0 # Asegurarse de que la columna exista con un valor por defecto
    aggregated_youtube_comments = pd.DataFrame(columns=['videoId', 'avg_comment_sentiment', 'total_comments']) # DataFrame vacío si no hay comentarios para agregar


# Eliminar columnas duplicadas o innecesarias de comments_df antes de la unión si ya existen en videos_df
# Esto es para evitar la colisión de '_id' si no es relevante, o si tienes otras columnas con nombres repetidos.
if '_id' in comments_df.columns:
    # Renombrar '_id' de comentarios para evitar conflicto con '_id' de videos al unir
    comments_df.rename(columns={'_id': 'comment_doc_id'}, inplace=True)
# Si quisieras eliminar otras columnas del comments_df que no necesitas en el final_df_youtube:
# comments_df = comments_df[['videoId', 'comment_sentiment_score', 'comment_doc_id']]




--- Estandarizando 'videoId' a string para la unión ---
Tipo de dato de 'videoId' en videos_df (después de estandarizar): object
Tipo de dato de 'videoId' en comments_df (después de estandarizar): object
------------------------------------------------------------

--- Realizando análisis de sentimiento en transcripciones de YouTube (desde videos_df) ---
Columna 'transcription_sentiment_score' añadida a videos_df.
       videoId                                      transcription  \
0  -h7oRpfHyyg  [Music] The last time we spoke about Lies of P...   
1  0XF4VBXPuhE  [Music] if you think you can change asa's fate...   
2  1BxOeeKU-Zs  and we're back with another episode of before ...   
3  1FE86W8YUtM  Dark Souls Dark Souls have you ever felt like ...   
4  1KKbekyFv4A  second row Shadows die twice is the hardest ga...   

   transcription_sentiment_score  
0                       0.002822  
1                       0.120281  
2                       0.145325  
3                       0.

In [37]:
# ... (Tu código anterior, incluyendo la lectura de MongoDB y el procesamiento de YouTube) ...


# --- Depuración justo antes de la unión principal de YouTube ---
print("\n--- INICIO DE DEPURACIÓN PARA LA UNIÓN DE YOUTUBE ---")

print("\nEstado de videos_df:")
print(f"¿videos_df está vacío? {videos_df.empty}")
if not videos_df.empty:
    print(f"Columnas de videos_df: {videos_df.columns.tolist()}")
    print(f"¿'videoId' en videos_df? {'videoId' in videos_df.columns}")
    if 'videoId' in videos_df.columns:
        print(f"Primeros 5 'videoId' en videos_df: {videos_df['videoId'].head().tolist()}")
        print(f"Tipo de dato de 'videoId' en videos_df: {videos_df['videoId'].dtype}")
    print(f"Número de filas en videos_df: {len(videos_df)}")
else:
    print("videos_df está vacío.")

print("\nEstado de aggregated_youtube_comments:")
print(f"¿aggregated_youtube_comments está vacío? {aggregated_youtube_comments.empty}")
if not aggregated_youtube_comments.empty:
    print(f"Columnas de aggregated_youtube_comments: {aggregated_youtube_comments.columns.tolist()}")
    print(f"¿'videoId' en aggregated_youtube_comments? {'videoId' in aggregated_youtube_comments.columns}")
    if 'videoId' in aggregated_youtube_comments.columns:
        print(f"Primeros 5 'videoId' en aggregated_youtube_comments: {aggregated_youtube_comments['videoId'].head().tolist()}")
        print(f"Tipo de dato de 'videoId' en aggregated_youtube_comments: {aggregated_youtube_comments['videoId'].dtype}")
    print(f"Número de filas en aggregated_youtube_comments: {len(aggregated_youtube_comments)}")
else:
    print("aggregated_youtube_comments está vacío.")

print("\n--- FIN DE DEPURACIÓN PARA LA UNIÓN DE YOUTUBE ---")


# --- Tu parte actual de unión de DataFrames de YouTube ---
print("\n--- Uniendo videos_df con métricas agregadas de comentarios por 'videoId' ---")
youtube_combined_df = pd.DataFrame() # Inicializar

# ... (El resto del código de unión que me has mostrado) ...


--- INICIO DE DEPURACIÓN PARA LA UNIÓN DE YOUTUBE ---

Estado de videos_df:
¿videos_df está vacío? False
Columnas de videos_df: ['_id', 'videoId', 'transcription', 'transcription_sentiment_score']
¿'videoId' en videos_df? True
Primeros 5 'videoId' en videos_df: ['-h7oRpfHyyg', '0XF4VBXPuhE', '1BxOeeKU-Zs', '1FE86W8YUtM', '1KKbekyFv4A']
Tipo de dato de 'videoId' en videos_df: object
Número de filas en videos_df: 91

Estado de aggregated_youtube_comments:
¿aggregated_youtube_comments está vacío? False
Columnas de aggregated_youtube_comments: ['videoId', 'avg_comment_sentiment', 'total_comments']
¿'videoId' en aggregated_youtube_comments? True
Primeros 5 'videoId' en aggregated_youtube_comments: ['-1bm1IgT5oA', '-9uMGePq3zw', '-boyUBHkJ-I', '-h7oRpfHyyg', '-qgOZDRDynw']
Tipo de dato de 'videoId' en aggregated_youtube_comments: object
Número de filas en aggregated_youtube_comments: 177

--- FIN DE DEPURACIÓN PARA LA UNIÓN DE YOUTUBE ---

--- Uniendo videos_df con métricas agregadas de com

In [38]:

print("\n--- Uniendo videos_df con métricas agregadas de comentarios por 'videoId' ---")
youtube_combined_df = pd.DataFrame() # Inicializar

if not videos_df.empty and not aggregated_youtube_comments.empty:
    youtube_combined_df = pd.merge(
        videos_df,
        aggregated_youtube_comments,
        on='videoId',
        how='left'
    )
    youtube_combined_df['avg_comment_sentiment'] = youtube_combined_df['avg_comment_sentiment'].fillna(0)
    youtube_combined_df['total_comments'] = youtube_combined_df['total_comments'].fillna(0)

    print(f"Se unieron videos y comentarios agregados de YouTube. Filas resultantes: {len(youtube_combined_df)}")
    print("Columnas del DataFrame combinado de YouTube:", youtube_combined_df.columns.tolist())
    print("Primeras 5 filas del DataFrame combinado de YouTube:\n", youtube_combined_df.head(5))
else:
    print("No se pudo unir videos_df y aggregated_youtube_comments porque uno o ambos están vacíos o les falta 'videoId'.")
    if not videos_df.empty:
        youtube_combined_df = videos_df.copy()
        youtube_combined_df['avg_comment_sentiment'] = 0
        youtube_combined_df['total_comments'] = 0
        print("youtube_combined_df contiene solo los datos de videos, sin métricas de comentarios.")




--- Uniendo videos_df con métricas agregadas de comentarios por 'videoId' ---
Se unieron videos y comentarios agregados de YouTube. Filas resultantes: 91
Columnas del DataFrame combinado de YouTube: ['_id', 'videoId', 'transcription', 'transcription_sentiment_score', 'avg_comment_sentiment', 'total_comments']
Primeras 5 filas del DataFrame combinado de YouTube:
                         _id      videoId  \
0  685e9d49164dd170f6984ef7  -h7oRpfHyyg   
1  685e9d49164dd170f6984ef9  0XF4VBXPuhE   
2  685e9d49164dd170f6984efb  1BxOeeKU-Zs   
3  685e9d49164dd170f6984efd  1FE86W8YUtM   
4  685e9d49164dd170f6984eff  1KKbekyFv4A   

                                       transcription  \
0  [Music] The last time we spoke about Lies of P...   
1  [Music] if you think you can change asa's fate...   
2  and we're back with another episode of before ...   
3  Dark Souls Dark Souls have you ever felt like ...   
4  second row Shadows die twice is the hardest ga...   

   transcription_sentiment_score

In [39]:

print("\n--- Guardando resultados combinados de YouTube en la colección 'yt_video_games_all' ---")

youtube_output_collection_name = 'yt_video_games_all'
yt_video_games_all_collection = db[youtube_output_collection_name]

# Opcional: Eliminar documentos existentes antes de insertar
yt_video_games_all_collection.delete_many({})
print(f"Colección '{youtube_output_collection_name}' vaciada (si contenía documentos).")

# Validar y guardar
if not youtube_combined_df.empty:
    print(f"Detectados {len(youtube_combined_df)} documentos de YouTube para insertar en '{youtube_output_collection_name}'.")
    youtube_records = youtube_combined_df.to_dict('records')
    try:
        yt_video_games_all_collection.insert_many(youtube_records)
        print(f"¡Éxito! Se insertaron {len(youtube_records)} documentos de YouTube combinados en '{youtube_output_collection_name}'.")
    except Exception as e:
        print(f"ERROR: Fallo al insertar documentos de YouTube combinados en '{youtube_output_collection_name}': {e}")
else:
    print(f"El DataFrame 'youtube_combined_df' está vacío. No se guardarán datos en '{youtube_output_collection_name}'.")

print("\nProceso de guardado de datos de YouTube completado.")




--- Guardando resultados combinados de YouTube en la colección 'yt_video_games_all' ---
Colección 'yt_video_games_all' vaciada (si contenía documentos).
Detectados 91 documentos de YouTube para insertar en 'yt_video_games_all'.
¡Éxito! Se insertaron 91 documentos de YouTube combinados en 'yt_video_games_all'.

Proceso de guardado de datos de YouTube completado.
